In [0]:
spark


In [0]:
job_skills = spark.read.csv(
    "/Volumes/workspace/default/job_market_data/job_skills.csv",
    header=True,
    inferSchema=True
)

job_skills.show(5)

+----------+---------+
|    job_id|skill_abr|
+----------+---------+
|3884428798|     MRKT|
|3884428798|       PR|
|3884428798|      WRT|
|3887473071|     SALE|
|3887465684|      FIN|
+----------+---------+
only showing top 5 rows


In [0]:
skills = spark.read.csv(
    "/Volumes/workspace/default/job_market_data/skills.csv",
    header=True,
    inferSchema=True
)

skills.show(5)


+---------+------------------+
|skill_abr|        skill_name|
+---------+------------------+
|      ART|      Art/Creative|
|     DSGN|            Design|
|     ADVR|       Advertising|
|     PRDM|Product Management|
|     DIST|      Distribution|
+---------+------------------+
only showing top 5 rows


In [0]:
salaries = spark.read.csv(
    "/Volumes/workspace/default/job_market_data/salaries.csv",
    header=True,
    inferSchema=True
)

salaries.show(5)


+---------+----------+----------+----------+----------+----------+--------+-----------------+
|salary_id|    job_id|max_salary|med_salary|min_salary|pay_period|currency|compensation_type|
+---------+----------+----------+----------+----------+----------+--------+-----------------+
|        1|3884428798|      NULL|      20.0|      NULL|    HOURLY|     USD|      BASE_SALARY|
|        2|3887470552|      25.0|      NULL|      23.0|    HOURLY|     USD|      BASE_SALARY|
|        3|3884431523|  120000.0|      NULL|  100000.0|    YEARLY|     USD|      BASE_SALARY|
|        4|3884911725|  200000.0|      NULL|   10000.0|    YEARLY|     USD|      BASE_SALARY|
|        5|3887473220|      35.0|      NULL|      33.0|    HOURLY|     USD|      BASE_SALARY|
+---------+----------+----------+----------+----------+----------+--------+-----------------+
only showing top 5 rows


In [0]:
print("Job skills rows:", job_skills.count())
print("Skills rows:", skills.count())

Job skills rows: 213768
Skills rows: 35


In [0]:
job_skills.printSchema()
skills.printSchema()


root
 |-- job_id: long (nullable = true)
 |-- skill_abr: string (nullable = true)

root
 |-- skill_abr: string (nullable = true)
 |-- skill_name: string (nullable = true)



In [0]:
job_skill_details = (
    job_skills
    .join(skills, on="skill_abr", how="inner")
)

job_skill_details.show(5)


+---------+----------+----------------+
|skill_abr|    job_id|      skill_name|
+---------+----------+----------------+
|     MRKT|3884428798|       Marketing|
|       PR|3884428798|Public Relations|
|      WRT|3884428798| Writing/Editing|
|     SALE|3887473071|           Sales|
|      FIN|3887465684|         Finance|
+---------+----------+----------------+
only showing top 5 rows


In [0]:
job_skill_details.printSchema()


root
 |-- skill_abr: string (nullable = true)
 |-- job_id: long (nullable = true)
 |-- skill_name: string (nullable = true)



In [0]:
from pyspark.sql.functions import count

top_skills = (
    job_skill_details
    .groupBy("skill_name")
    .agg(count("*").alias("demand_count"))
    .orderBy("demand_count", ascending=False)
)

top_skills.show(10)


+--------------------+------------+
|          skill_name|demand_count|
+--------------------+------------+
|Information Techn...|       26137|
|               Sales|       22475|
|          Management|       20861|
|       Manufacturing|       18185|
|Health Care Provider|       17369|
|Business Development|       14290|
|         Engineering|       13009|
|               Other|       12608|
|             Finance|        8540|
|           Marketing|        5525|
+--------------------+------------+
only showing top 10 rows


In [0]:
top_skills.createOrReplaceTempView("top_skills")

In [0]:
%sql
SELECT skill_name, demand_count
FROM top_skills
ORDER BY demand_count DESC
LIMIT 10;


skill_name,demand_count
Information Technology,26137
Sales,22475
Management,20861
Manufacturing,18185
Health Care Provider,17369
Business Development,14290
Engineering,13009
Other,12608
Finance,8540
Marketing,5525


Databricks visualization. Run in Databricks to view.

In [0]:
top_skills.limit(5).toPandas()


,skill_name,demand_count
0,Information Technology,26137
1,Sales,22475
2,Management,20861
3,Manufacturing,18185
4,Health Care Provider,17369


In [0]:
top_skills.write.mode("overwrite").csv(
    "/Volumes/workspace/default/job_market_data/top_skills_output"
)


In [0]:
from pyspark.sql.functions import when

skill_tiers = top_skills.withColumn(
    "skill_tier",
    when(top_skills.demand_count >= 15000, "Core Skill")
    .when(top_skills.demand_count >= 10000, "Important Skill")
    .otherwise("Optional / Niche Skill")
)

skill_tiers.show(10, truncate=False)

+----------------------+------------+----------------------+
|skill_name            |demand_count|skill_tier            |
+----------------------+------------+----------------------+
|Information Technology|26137       |Core Skill            |
|Sales                 |22475       |Core Skill            |
|Management            |20861       |Core Skill            |
|Manufacturing         |18185       |Core Skill            |
|Health Care Provider  |17369       |Core Skill            |
|Business Development  |14290       |Important Skill       |
|Engineering           |13009       |Important Skill       |
|Other                 |12608       |Important Skill       |
|Finance               |8540        |Optional / Niche Skill|
|Marketing             |5525        |Optional / Niche Skill|
+----------------------+------------+----------------------+
only showing top 10 rows


In [0]:
skill_tiers.createOrReplaceTempView("skill_tiers")


In [0]:
%sql
SELECT skill_tier, COUNT(*) AS total_skills
FROM skill_tiers
GROUP BY skill_tier;


skill_tier,total_skills
Core Skill,5
Optional / Niche Skill,27
Important Skill,3


Databricks visualization. Run in Databricks to view.